In [1]:
### This code is for generating Protein Content Networks (PCN), degree distributions and calculating dij values
    # from the deep metaproteomics dataset using the protein-peptide bridge approach.

### About original datafiles in the folder \1_database_search_results:
    ## MetaLab_peptide_n.xlsx, proteinGroups.txt, peptides.txt, function.csv are generated by database search using MetaLab
    ## function1.txt and proteinGroups_top1.txt is a short version of the function and proteinGroups tables
    ## by keeping the first protein of each protein group.

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
print("Current Working Directory " , os.getcwd())

Current Working Directory  /Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised


In [4]:
#Step1 match peptide IDs to taxon table
# Read MetaLab_peptide_n.xlsx tables, there were two generated by the MetaLab search, need to merge.
taxon_table_1 = pd.read_excel('1_database_search_results/MetaLab_peptide_1.xlsx', sheet_name='peptide list') # it takes a few minutes to read the tables
taxon_table_2 = pd.read_excel('1_database_search_results/MetaLab_peptide_2.xlsx', sheet_name='peptide list')
taxon_table = pd.concat([taxon_table_1, taxon_table_2], axis=0)
# Read peptides.txt that are generated by MetaLab
peptide_table = pd.read_table('1_database_search_results/peptides.txt', sep = '\t', low_memory=False)
# Obtain only the peptide ids and sequences form peptide_table
peptide_ID_match = peptide_table[['Sequence', 'id']]
# Remove the id from taxon_table
# Match real ids to the table
taxon_table_drop = taxon_table.drop(columns='Peptide id')
merge_table = taxon_table_drop.merge(peptide_ID_match, on='Sequence')
# Generate peptide id - Sequence - taxon table
peptIDe_to_taxon = pd.DataFrame(merge_table, columns = ['id', 'Sequence', 'LCA', 'Rank', 'Superkingdom', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species'])
peptIDe_to_taxon.to_csv('2_data_processing/1_PCN/PCN_generation/peptIDe_to_taxon_MP_deep.csv')

/Users/leyuan/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/leyuan/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
#Step2 match proteinGroups to taxon table according bridged by peptide IDS
# Need to obtain a dataframe of ProteinGroups_and_peptide_IDs
# read proteinGroups.txt, and only keep two columns -- protein group top1 names and peptides IDs
proteinGroups_table = pd.read_table('1_database_search_results/proteinGroups.txt', sep = '\t', low_memory=False)
proteinGroups_ids = proteinGroups_table[['Protein IDs', 'Peptide IDs']]
proteinGroups_ids_split = pd.concat([proteinGroups_ids["Protein IDs"].str.split("\;", n=1, expand = True).drop(columns=1).
                                    rename(columns = {0 : 'Protein IDs'}), proteinGroups_ids["Peptide IDs"].str.split("\;", expand = True)], axis = 1)
long_matches = pd.melt(proteinGroups_ids_split, id_vars=['Protein IDs'], value_name='id').drop(columns='variable').dropna().\
               sort_values('Protein IDs', inplace = False).reset_index().drop(columns='index')
long_matches['id'] = long_matches['id'].astype('int64')
pro_pep_tax = long_matches.merge(peptIDe_to_taxon, on = 'id')
pro_pep_tax.to_csv('2_data_processing/1_PCN/PCN_generation/pro_pep_tax_MP_deep.csv')

In [6]:
#Step3 match proteinGroups to function
#read unique genus matches table.
#Used Excel to obtain the unique matches (for details, see Supplementary Note 1), and saved the table as 'pro_pep_tax_MP_deep_unique.csv'
protein_taxon_table = pd.read_table('2_data_processing/1_PCN/PCN_generation/pro_pep_tax_MP_deep_unique.csv', sep = ',')
#read function table, and combine with protein group table, then unique genus matches table
## The function table contains a column that is a combination of KEGG and COG, 
## first proteins were matched to KEGG using GhostKOALA. The remaining proteins 
## without a KEGG ko match was complemented with a COG from the functional output of MetaLab.
function_table = pd.read_table('1_database_search_results/function_COG_KEGG.txt', sep = '\t', low_memory=False).rename(columns = {'Name': 'Protein IDs'})
protein_top1_table = pd.read_table('1_database_search_results/proteinGroups1.txt', sep = '\t', low_memory=False)
function_table_top1 = function_table.merge(protein_top1_table, on = 'Protein IDs')
merge_pro_tax_fun = protein_taxon_table.merge(function_table_top1, on = 'Protein IDs')
merge_pro_tax_fun.to_csv('2_data_processing/1_PCN/PCN_generation/Full_table_pro_tax_fun_MP_deep_KEGG_COG.csv')

In [7]:
#Step4 generate PCNs based on Functions in the funtion table
pro_gen_cog = pd.DataFrame(merge_pro_tax_fun, columns = ['Protein IDs', 'Genus', 'KEGG_COG'])
proteinGroups_table = pd.read_table('1_database_search_results/proteinGroups.txt', sep = '\t', low_memory=False)
proteinGroups_intensity = pd.concat([proteinGroups_table["Protein IDs"].str.split("\;", n=1, expand = True).drop(columns=1).rename(columns = {0 : 'Protein IDs'}),
                                     proteinGroups_table.filter(regex='Intensity HM')], axis = 1)
pro_gen_cog_int = pro_gen_cog.merge(proteinGroups_intensity, on = 'Protein IDs')
pro_gen_cog_int.columns=pro_gen_cog_int.columns.str.replace('Intensity','')
pro_gen_cog_int.to_csv('2_data_processing/1_PCN/PCN_generation/Full_table_pro_tax_fun_KEGG_COG_for_PCN.csv')
#Use loop to generate and save PCN for each sample
for x in pro_gen_cog_int.columns[3:7]:
    # get columns for this specific column
    pro_gen_cog_sam = pd.DataFrame(pro_gen_cog_int, columns = ['Genus', 'KEGG_COG', x])
    # print(pro_gen_cog_sam.columns)
    # do sum by groups
    pro_gen_cog_sam_sum = pro_gen_cog_sam.groupby(['Genus', 'KEGG_COG']).sum().reset_index()
    # reshape to wide table
    pro_gen_cog_sam_sum_wide = pro_gen_cog_sam_sum.pivot(index='KEGG_COG', columns='Genus', values= x )
    # write to tables
    pro_gen_cog_sam_sum_wide.to_csv('2_data_processing/1_PCN/PCN_tables_raw/PCN_KEGG_COG' + x + '.csv')
    print(x)
#endregion

 HM454
 HM455
 HM466
 HM503


In [8]:
#Step 5. Generate 0 1 PCN tables
### Batch process files - from genus to COG tables, generate an overall PCN and individual sample PCNs ########
def findfile(path, tagstr):
    #setup an empty dataframe with same index and column names as all tables
    path = os.path.abspath(path)
    for x in os.listdir(path):
        fulldir = os.path.join(path, x)  # join to absolute path
        if os.path.isfile(fulldir):  # file, match -> print
            # if os.path.split(fulldir)[1]==tagstr   # look for the designated file
            if tagstr in os.path.splitext(fulldir)[1]:  # look for files containing the key word
                print(os.path.join(os.path.abspath(fulldir), fulldir))
                PCN_table_single = pd.read_table(fulldir, delimiter=",").set_index(['KEGG_COG']).fillna(0).T
                if_detected = PCN_table_single.astype('float') > 0
                PCN_table_single['num_detected'] = if_detected.sum(axis=1)
                PCN_table_single_filtered = PCN_table_single.loc[(PCN_table_single.sum(axis=1) != 0), (PCN_table_single.sum(axis=0) != 0)].\
                    sort_values(by ='num_detected', ascending=False).drop(columns='num_detected') #remove empty columns for individual files
                PCN_table_single_filtered = PCN_table_single_filtered.astype(
                    'float') > 0  # replace values with presence or not (1 or 0), it will give a True or False value
                PCN_table_single_filtered = PCN_table_single_filtered.astype('float')  # replace True or False with 1 or 0
                PCN_table_single_filtered_sum = pd.DataFrame(PCN_table_single_filtered.sum(axis=0)).rename(columns={0: 'sum_num'}).T
                PCN_table_single_filtered_sum_col = pd.concat([PCN_table_single_filtered, PCN_table_single_filtered_sum]).sort_values(by='sum_num', axis=1,
                                                                                            ascending=False).drop(
                    index='sum_num') # get sorted PCN for each individual sample
                PCN_table_single_filtered_sum_col.to_csv('2_data_processing/1_PCN/PCN_tables_0_1/' + os.path.splitext(x)[0] + '_PCN_0_1' + '.csv')

findfile('2_data_processing/1_PCN/PCN_tables_raw/', '.csv')

/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_raw/PCN_KEGG_COG HM503.csv
/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_raw/PCN_KEGG_COG HM466.csv
/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_raw/PCN_KEGG_COG HM455.csv
/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_raw/PCN_KEGG_COG HM454.csv


In [9]:
## Part 2. Degree distributions
# Degree distributions
def findfile(path, tagstr):
    path = os.path.abspath(path)

    for x in os.listdir(path):

        fulldir = os.path.join(path, x)  # join to absolute path

        if os.path.isfile(fulldir):  # file, match -> print
            # if os.path.split(fulldir)[1]==tagstr   # look for the designated file
            if tagstr in os.path.splitext(fulldir)[1]:  # look for files containing the key word
                print(os.path.join(os.path.abspath(fulldir), fulldir))

                ######## Calculation starts here ###############################
                origin_data = pd.read_table(fulldir, delimiter=",", index_col=0)           #read a PCN_0_1 table

                taxon_degree_distribution = pd.DataFrame()
                COG_degree_distribution = pd.DataFrame()

                # calculate number of nodes k degree
                taxon_degree_distribution['k'] = origin_data.sum(axis=1)
                COG_degree_distribution['k'] = origin_data.sum(axis=0)

                # calculate pk fraction of nodes
                taxon_pk = pd.DataFrame(columns=['k'])
                taxon_pk['pk'] = taxon_degree_distribution['k'].value_counts() / len(taxon_degree_distribution.index)
                taxon_pk['k'] = taxon_pk.index
                taxon_degree_distribution = taxon_pk.merge(taxon_degree_distribution, on='k')

                COG_pk = pd.DataFrame(columns=['k'])
                COG_pk['pk'] = COG_degree_distribution['k'].value_counts() / len(COG_degree_distribution.index)
                COG_pk['k'] = COG_pk.index
                COG_degree_distribution = COG_pk.merge(COG_degree_distribution, on='k')

                taxon_degree_distribution.to_csv('2_data_processing/2_Degree_distribution/Taxon/'+ x)
                COG_degree_distribution.to_csv('2_data_processing/2_Degree_distribution/Function/'+ x)


findfile('2_data_processing/1_PCN/PCN_tables_0_1', '.csv')

/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_0_1/PCN_KEGG_COG HM503_PCN_0_1.csv
/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_0_1/PCN_KEGG_COG HM455_PCN_0_1.csv
/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_0_1/PCN_KEGG_COG HM466_PCN_0_1.csv
/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_0_1/PCN_KEGG_COG HM454_PCN_0_1.csv


In [ ]:
### Part 3. deep MP dataset, all genera, functional distance ###
# the results will be used for FR calculation, but not for dij visualization
def findfile(path, tagstr):
    path = os.path.abspath(path)

    for x in os.listdir(path):

        fulldir = os.path.join(path, x)  # join to absolute path

        if os.path.isfile(fulldir):  # file, match -> print
            # if os.path.split(fulldir)[1]==tagstr   # look for the designated file
            if tagstr in os.path.splitext(fulldir)[1]:  # look for files containing the key word
                print(os.path.join(os.path.abspath(fulldir), fulldir))

                ######## Calculation starts here ###############################
                origin_data = pd.read_table(fulldir, delimiter=",", index_col=0).fillna(0) #read a PCN table and fill nan with zero

                # Protein content network
                PCN = origin_data # PCN without any normalization
                PCN = PCN[PCN.columns[PCN.sum()>0]] # remove genus column if sum to zero
                PCN_taxa_norm = PCN.apply(lambda x: x/sum(x), axis = 0).fillna(0) # Proteomic content network / normalized to each taxon

                PCN_taxa_norm_T = PCN_taxa_norm.T
                if_detected = PCN_taxa_norm_T > 0  # turn PCN numbers to 0 or 1
                PCN_taxa_norm_T['num_detected'] = if_detected.sum(axis=1)  # obtain num_detected,
                PCN_taxa_norm_all = PCN_taxa_norm_T.fillna(0).drop(columns='num_detected').T

                # create an empty dataframe
                dij_matrix_norm = pd.DataFrame()
                dij_list_norm = pd.DataFrame(
                    index=range((len(PCN_taxa_norm_all.columns)) * (len(PCN_taxa_norm_all.columns) - 1) // 2),
                    columns=['dij_list'])
                dij_list_name_norm = pd.DataFrame(
                    index=range((len(PCN_taxa_norm_all.columns)) * (len(PCN_taxa_norm_all.columns) - 1) // 2),
                    columns=['VS'])
                dij_num_norm = 0
                # getting a dij between any two bacteria genera
                for i in range(0, len(PCN_taxa_norm_all.columns)):
                    Gi = PCN_taxa_norm_all.iloc[:, i]
                    print(i)
                    for j in range(0, len(PCN_taxa_norm_all.columns)):
                        Gj = PCN_taxa_norm_all.iloc[:, j]
                        Gij = pd.concat([Gi, Gj], axis=1)
                        dij = 1 - sum(Gij.apply(lambda x: min(x), axis=1)) / sum(Gij.apply(lambda x: max(x), axis=1))
                        if i > j:
                            dij_matrix_norm.at[Gij.columns.values[0], Gij.columns.values[1]] = dij
                            dij_list_norm.at[dij_num_norm, 'dij_list'] = dij
                            dij_num_norm = dij_num_norm + 1
                            dij_list_name_norm.at[dij_num_norm-1] = Gij.columns.values[0] + '_vs_' + Gij.columns.values[1]
                dij_list_summary_norm = pd.concat([dij_list_name_norm, dij_list_norm], axis=1)
                dij_matrix_norm.to_csv('2_data_processing/3_PCN_Functional_distance/all/norm/matrix/all_norm_dij_matrix_'+ x)
                dij_list_summary_norm.to_csv('2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_'+ x)
                # endregion
findfile('2_data_processing/1_PCN/PCN_tables_raw', '.csv')


/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_raw/PCN_KEGG_COG HM503.csv
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
/Users/leyuan/Documents/lab/paper/FR/NatMethods/V2/codes/FRp_codes_revised/2_data_processing/1_PCN/PCN_tables_raw/PCN_KEGG_COG HM466.csv
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32

In [23]:
# Calculates functional redundancy following this equation:
# FR = ∑(N,i=1)∑(N,j=1)(1-dij)pipj

## loop for all samples - deep metaproteomics
######## Batch process files - calculating Dij for each sample ########

genus_table_original = pd.read_table("1_database_search_results/Taxonomy_table_genus_level_metaproteomics.csv",
                                     delimiter=",", index_col=0)
genus_table_p_matrix = genus_table_original.div(genus_table_original.sum(axis=0), axis=1)

## loop for all samples - deep metaproteomics
######## Batch process files - calculating Dij for each sample ########
def findfile(path, tagstr):
    Record_FR = pd.DataFrame()
    Record_FR_norm = pd.DataFrame()
    count = 0

    path = os.path.abspath(path)

    for x in os.listdir(path):

        fulldir = os.path.join(path, x)  # join to absolute path

        if os.path.isfile(fulldir):  # file, match -> print
            # if os.path.split(fulldir)[1]==tagstr   # look for the designated file
            if tagstr in os.path.splitext(fulldir)[1]:  # look for files containing the key word
                print(os.path.join(os.path.abspath(fulldir), fulldir))

                ######## Calculation starts here ###############################
                dij_table_data = pd.read_table(fulldir, delimiter=",", index_col=0)
                dij_table_data_indexed = pd.concat(
                    [dij_table_data, dij_table_data["VS"].str.split("_vs_", n=1, expand=True)], axis=1).rename(
                    columns={0: "i", 1: "j"})
                # index to the sample column in the genus_table_p_matrix
                genus_table_p_sample_i = genus_table_p_matrix[[x[-9:-4]]].rename(columns={x[-9:-4]: 'Abundance'})
                genus_table_p_sample_j = genus_table_p_matrix[[x[-9:-4]]].rename(columns={x[-9:-4]: 'Abundance'})

                # assign pi and pj values to the data table
                genus_table_p_sample_i = genus_table_p_sample_i.reset_index().rename(columns={'Name': "i"})
                genus_table_p_sample_j = genus_table_p_sample_j.reset_index().rename(columns={'Name': "j"})
                dij_table_trial_indexed_p = dij_table_data_indexed.merge(genus_table_p_sample_i, on='i').rename(
                    columns={'Abundance': 'pi'})
                dij_table_trial_indexed_p = dij_table_trial_indexed_p.merge(genus_table_p_sample_j, on='j').rename(
                    columns={'Abundance': 'pj'})

                # Calculate (1-dij)pipj
                FR = 0
                GSI = 0
                for x in dij_table_trial_indexed_p.index:
                    FR_ij = (1 - dij_table_trial_indexed_p.loc[x, 'dij_list']) * dij_table_trial_indexed_p.loc[
                        x, 'pi'] * dij_table_trial_indexed_p.loc[x, 'pj']
                    FR = FR + FR_ij * 2
                    # Taxonomic diversity: Gini-Simpson index
                    GSI_ij = dij_table_trial_indexed_p.loc[x, 'pi'] * dij_table_trial_indexed_p.loc[x, 'pj']
                    GSI = GSI + GSI_ij * 2
                print(FR)
                print(GSI)
                Record_FR.loc[count, 0] = fulldir[-9:-4]
                Record_FR.loc[count, 1] = FR
                Record_FR.loc[count, 2] = FR/GSI # Functional redundancy normalized by taxonomic diveristy
                Record_FR.loc[count, 3] = GSI
                Record_FR.loc[count, 4] = GSI-FR
                count = count + 1

    Record_FR.rename(columns={0: "Sample", 1: "FR", 2: "nFR", 3: "TD", 4: "FD"}).to_csv('2_data_processing/4_Functional_redundancy/' + 'Record_FR_PCN.csv')


findfile('2_data_processing/3_PCN_Functional_distance/all/norm/list', '.csv')

/Users/leyuan/Documents/lab/data_analysis/20210310_FR_study/NC_revise_20221228/Ultra_deep_MetaPro_IQ/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_PCN_KEGG_COG HM503.csv
0.03445541738848285
0.39772851423029276
/Users/leyuan/Documents/lab/data_analysis/20210310_FR_study/NC_revise_20221228/Ultra_deep_MetaPro_IQ/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_PCN_KEGG_COG HM466.csv
0.10212832444431968
0.6682941421468552
/Users/leyuan/Documents/lab/data_analysis/20210310_FR_study/NC_revise_20221228/Ultra_deep_MetaPro_IQ/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_PCN_KEGG_COG HM454.csv
0.12424754019234395
0.7478833576642722
/Users/leyuan/Documents/lab/data_analysis/20210310_FR_study/NC_revise_20221228/Ultra_deep_MetaPro_IQ/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_PCN_KEGG_COG HM455.csv
0.12456017177383634
0.7169178609450153
